In [1]:
import os
import glob
import re
import pandas

import iqplot
import colorcet

import bokeh.io
import bokeh.plotting

bokeh.io.output_notebook()

Loading BokehJS ...

In [2]:
regex = re.compile("grant_([0-9]{4}).csv")

In [3]:
dfs = []

for file in os.listdir("data"):
    
    matches = regex.search(file)
    
    if matches is None:
        continue
        
    year = matches.group(1)
    
    df = pandas.read_csv(os.path.join("data", file), comment="#")
    
    if "yearband" in df.columns:
        df["year"] = df["yearband"]
        if max(df["year"]) < 100:
            df["year"] += 1900
        df = df.drop("yearband", axis=1)
    else:
        df["year"] = int(year)
        
    for column in df.columns:
        if column.find("depth") != -1:
            df["beak depth (mm)"] = df[column]
            df = df.drop(column, axis=1)
        elif column.find("length") != -1:
            df["beak length (mm)"] = df[column]
            df = df.drop(column, axis=1)
    
    dfs.append(df)

In [4]:
COMPLETE_CSV_FILE_PATH = os.path.join("data", "grant_complete_2.csv")

df = dfs[0].append(dfs[1:])
df.drop_duplicates(["band", "year"], inplace=True)
df.to_csv(COMPLETE_CSV_FILE_PATH, index=False)

In [5]:
df[df["band"] == 19028]

,band,species,year,beak length (mm),beak depth (mm)
1,19028,fortis,2012,12.5,8.9


In [7]:
df = dfs[0].append(dfs[1:])
df

,band,species,year,beak length (mm),beak depth (mm)
0,19022,fortis,2012,10.00,8.50
1,19028,fortis,2012,12.50,8.90
2,19032,fortis,2012,9.30,7.50
3,19041,fortis,2012,10.30,9.60
4,19044,fortis,2012,11.00,9.20
...,...,...,...,...,...
84,20224,scandens,1973,15.65,9.95
85,20245,scandens,1973,14.05,9.55
86,20254,scandens,1973,13.85,9.15
87,20259,scandens,1973,14.95,10.45


In [ ]:
df.drop_duplicates(["band", "species"])

In [92]:
df.groupby(["species", "year"]).describe()["beak length (mm)"]

count       mean       std    min      25%     50%      75%  \
species  year                                                                
fortis   1973   73.0  10.655479  0.743843   9.05  10.1500  10.650  11.1500   
         1975  312.0  10.562821  0.673305   8.70  10.1000  10.575  11.0000   
         1987  787.0  10.941576  0.770277   8.83  10.4000  11.000  11.4400   
         1991  509.0  10.852515  0.785460   8.80  10.3700  10.900  11.3900   
         2012  121.0  10.517355  0.807741   9.00  10.0000  10.400  10.9000   
scandens 1973   16.0  14.568750  0.692068  13.15  14.0500  14.650  15.0750   
         1975   87.0  14.120920  0.754789  12.80  13.5000  14.000  14.7400   
         1987  156.0  14.197436  0.862055  10.75  13.8375  14.255  14.8175   
         1991  112.0  13.934464  1.059413  10.30  13.4825  14.120  14.7000   
         2012  126.0  13.428333  0.718525  11.40  13.0000  13.400  13.9000   

                 max  
species  year         
fortis   1973  12.65  
         1975  12.20  
         1987  14.07  
         1991  14.07  
         2012  12.90  
scandens 1973  15.65  
         1975  15.94  
         1987  16.00  
         1991  15.80  
         2012  15.20

In [101]:
figure = iqplot.box(
    data=df,
    q="beak length (mm)",
    cats=["species", "year"],
    q_axis="y"
)

bokeh.io.show(figure)

In [102]:
figure = iqplot.box(
    data=df,
    q="beak depth (mm)",
    cats=["species", "year"],
    q_axis="y"
)

bokeh.io.show(figure)

In [111]:
def scatter(data, cat, x, y):
    
    figure = bokeh.plotting.figure(
        x_axis_label=x,
        y_axis_label=y
    )
    
    colors = colorcet.palette["glasbey_category10"]
    
    for i, category in enumerate(data[cat].unique()):
        
        category_mask = data[cat] == category
    
        figure.circle(
            source=df[category_mask],
            x=x,
            y=y,
            color=colors[i],
            legend_label=str(category)
        )
    
    bokeh.io.show(figure)

In [112]:
scatter(df, "year", "beak length (mm)", "beak depth (mm)")

In [116]:
df["beak area (mm^2)"] = df["beak depth (mm)"] * df["beak length (mm)"]
df["beak depth:length ratio"] = df["beak depth (mm)"] / df["beak length (mm)"]

In [115]:
figure = iqplot.box(
    data=df,
    q="beak area (mm^2)",
    cats=["species", "year"],
    q_axis="y"
)

bokeh.io.show(figure)

In [117]:
figure = iqplot.box(
    data=df,
    q="beak depth:length ratio",
    cats=["species", "year"],
    q_axis="y"
)

bokeh.io.show(figure)